In [1]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
import yfinance as yf
import json
from openai import OpenAI


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    ticker = ddg.run(f"Ticker symbol of {company_name}")
    print(f"Retrieved ticker: {ticker}")
    return ticker


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yf.Ticker(ticker)
    income_statement = stock.financials.to_json()
    print(f"Retrieved income statement for {ticker}")
    return income_statement


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet.to_json()
    print(f"Retrieved balance sheet for {ticker}")
    return balance_sheet


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yf.Ticker(ticker)
    stock_performance = stock.history(period="3mo").to_json()
    print(f"Retrieved stock performance for {ticker}")
    return stock_performance


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company, returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e., AAPL), returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e., AAPL), returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e., AAPL), returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Investor Assistant v2.8",
    instructions="You help users perform research on publicly traded companies to decide if they should buy the stock or not.",
    tools=functions,
    model="gpt-4o",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to know if Extra Space Storage stock is a good buy.",
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs,
    )

In [6]:
get_run(run.id, thread.id).status

'requires_action'

In [3]:
get_messages(thread.id)

user: I want to know if Extra Space Storage stock is a good buy.


In [4]:
get_tool_outputs(run.id, thread.id)

Calling function: get_ticker with arg {"company_name":"Extra Space Storage"}
Retrieved ticker: Get a real-time Extra Space Storage Inc. (EXR) stock price quote with breaking news, financials, statistics, charts and more. ... Ticker Symbol EXR. Full Company Profile. Financial Performance. In 2023, EXR's revenue was $2.56 billion, an increase of 33.06% compared to the previous year's $1.92 billion. Earnings were $803.20 million, a decrease ... A high-level overview of Extra Space Storage Inc. (EXR) stock. Stay up to date on the latest stock price, chart, news, analysis, fundamentals, trading and investment tools. Stock analysis for Extra Space Storage Inc (EXR:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. The latest Extra Space Storage stock prices, stock quotes, news, and EXR history to help you invest and trade smarter. ... P/E ratio (year end quote, diluted EPS) 33.83 22.97 36.60 31.21 ... Extra Space Storage's stock was 

[{'output': "Get a real-time Extra Space Storage Inc. (EXR) stock price quote with breaking news, financials, statistics, charts and more. ... Ticker Symbol EXR. Full Company Profile. Financial Performance. In 2023, EXR's revenue was $2.56 billion, an increase of 33.06% compared to the previous year's $1.92 billion. Earnings were $803.20 million, a decrease ... A high-level overview of Extra Space Storage Inc. (EXR) stock. Stay up to date on the latest stock price, chart, news, analysis, fundamentals, trading and investment tools. Stock analysis for Extra Space Storage Inc (EXR:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. The latest Extra Space Storage stock prices, stock quotes, news, and EXR history to help you invest and trade smarter. ... P/E ratio (year end quote, diluted EPS) 33.83 22.97 36.60 31.21 ... Extra Space Storage's stock was trading at $160.33 on January 1st, 2024. Since then, EXR stock has decreased by 3.